In [ ]:

import nest_asyncio
import asyncio
import logging
from telethon import TelegramClient, events, Button
from openai import OpenAI

# Enable nest_asyncio to avoid event loop conflicts in Jupyter or Colab
nest_asyncio.apply()

# Define necessary credentials
api_id = "21403141"
api_hash = "74e9c96971cd422c1ba9878a13ea3db1"
bot_token = "7847748804:AAGrnFtZLY9GH6K1Rk0Mo_fX5V0bLt0dcr4"

# Initialize the client
client = TelegramClient('bot', api_id, api_hash)

# Configure OpenRouter API
openrouter_client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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"
)

# Define main function
async def main():
    await client.start(bot_token=bot_token)

    # Handle /start command
    @client.on(events.NewMessage(pattern='/start'))
    async def start_handler(event):
        user = await client.get_entity(event.sender_id)
        first_name = user.first_name if user.first_name else "there"

        await event.respond(
            f"👋 Hello {first_name}! I am EdgeAI from HyperVision. How can I assist you today? 🤖",
            buttons=[
                [Button.url("🌐 Visit our website", "https://example.com")],
                [Button.url("ℹ️ Learn more about me", "https://example.com/info")],
                [Button.url("📢 Join our community", "https://t.me/your_channel")]
            ]
        )
        logging.info(f'Start command received from {event.sender_id}')

    # Handle /info command
    @client.on(events.NewMessage(pattern='/info'))
    async def info_handler(event):
        await event.respond("🤖 I am EdgeAI from HyperVision, developed by Wail Achouri. 🎯")
        logging.info(f'Info command received from {event.sender_id}')

    # Handle /help command
    @client.on(events.NewMessage(pattern='/help'))
    async def help_handler(event):
        help_text = (
            "🛠️ **Available Commands:**\n"
            "/start - Start the bot 🚀\n"
            "/help - Get help 📖\n"
            "/info - Learn more about me ℹ️\n"
            "/generate_image description - Generate an AI image 🎨"
        )
        await event.respond(help_text)
        logging.info(f"Help command received from {event.sender_id}")

    # Handle AI-generated images
    @client.on(events.NewMessage(pattern='/generate_image (.+)'))
    async def image_handler(event):
        user = await client.get_entity(event.sender_id)
        first_name = user.first_name if user.first_name else "Friend"
        prompt = event.pattern_match.group(1)

        await event.respond(f"🎨 Generating your image, {first_name}... Please wait. ⏳")

        try:
            image_generation = await asyncio.to_thread(
                openrouter_client.images.generate,
                model="google/gemini-2.5-pro-exp-03-25:free",
                prompt=prompt,
                n=1,
                size="1024x1024"
            )

            image_url = image_generation.data[0].url
            await client.send_file(event.chat_id, image_url, caption=f"✨ Here is your image, {first_name}! 🎨")

        except Exception as e:
            logging.error(f"Error generating image: {e}")
            await event.respond("⚠️ Sorry, I couldn't generate the image. Please try again later.")

    # Handle user messages
    @client.on(events.NewMessage)
    async def keyword_responder(event):
        message = event.text.lower()
        if message.startswith('/'):
            return

        user = await client.get_entity(event.sender_id)
        first_name = user.first_name if user.first_name else "Friend"

        try:
            instructions = """
            You are EdgeAI, an advanced AI developed by HyperVision and created by Wail Achouri.
            Your responses should be short, fast, and include relevant emojis.
            Always personalize responses using the user's first name if available.
            """

            completion = await asyncio.to_thread(
                openrouter_client.chat.completions.create,
                extra_headers={
                    "HTTP-Referer": "<YOUR_SITE_URL>",
                    "X-Title": "<YOUR_SITE_NAME>",
                },
                extra_body={},
                model="google/gemini-2.5-pro-exp-03-25:free",
                messages=[
                    {"role": "system", "content": instructions},
                    {"role": "user", "content": f"The user's name is {first_name} and they asked: {message}"}
                ]
            )

            response = completion.choices[0].message.content

            await event.respond(response + " 😊")
            logging.info(f"Message received from {event.sender_id}: {event.text}")

        except Exception as e:
            logging.error(f"Error while fetching response: {e}")
            await event.respond("⚠️ Sorry, I couldn't process your request right now.")

    await client.run_until_disconnected()

# Run the main function
await main()

ERROR:root:Error while fetching response: Error code: 401 - {'error': {'message': 'No auth credentials found', 'code': 401}}
ERROR:root:Error generating image: Error code: 405
ERROR:root:Error while fetching response: Error code: 401 - {'error': {'message': 'No auth credentials found', 'code': 401}}
ERROR:root:Error while fetching response: Error code: 401 - {'error': {'message': 'No auth credentials found', 'code': 401}}
ERROR:root:Error while fetching response: Error code: 401 - {'error': {'message': 'No auth credentials found', 'code': 401}}


In [3]:
!pip install telethon

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.9/715.9 kB 18.5 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26346 sha256=64a7e76c35ce78910788b15c415181b8763232955721437b5ecea8a32d931e0c
  Stored in directory: /root/.cache/pip/wheels/4e/52/33/010d0843550bffb6a591b11629070ae140c0ad4f53e68a3bd3
Successfully built pyaes
